In [99]:
#pip install moviepy
import moviepy
from moviepy.editor import VideoFileClip
from pathlib import Path
import pandas as pd
import numpy as np
import random
import os

In [100]:
# variables
DLC_csvname = 'DLC_resnet50_Thesis project actualSep7shuffle1_700000' #change to namepart that does not correspond to avi.
header_use = [1] # to select the header row later
coordinates_use = [2] # to select the coordinates row later

In [101]:
#specify starting directory as the one we are working in now with this Jupyter file
input_dir = Path.cwd()

In [102]:
# extract header before everything. Not the most efficient but then the header is correct from the beginning.
pathheader = random.choice(list(input_dir.rglob("*.avi*")))
fileheader, file_ex = os.path.splitext(pathheader)
csv_header = fileheader + DLC_csvname + '.csv'

# extract header
header = pd.read_csv(csv_header,skiprows=lambda x: x not in header_use)
coordinates = pd.read_csv(csv_header,skiprows=lambda x: x not in coordinates_use)

# combine 
headertotal = [x+str(y) for x,y in zip(header,coordinates)]
headertotal.append('videoname')

In [103]:
# creating folder for downsampled and completed pre-processed data
os.makedirs("Downsampled_data", exist_ok=True)
os.makedirs("pre-processed data", exist_ok=True)

In [104]:
#create big dataframe
df_big = pd.DataFrame(columns = headertotal) #run everything once, otherwise 

# loop through video paths and retrieve metadata
for path in list(input_dir.rglob("*.avi*")):
    
    # retrieve frames per second metadata from video
    fps = int(VideoFileClip(str(path)).fps)
    
    #select corresponding csv to video to get correct data, given that it is in the same folder as the .avi
    filename, fileextension = os.path.splitext(path)
    csv = filename + DLC_csvname + '.csv'
    
    #load data
    df_temp = pd.read_csv(csv, header=None, names = headertotal,skiprows=3)
    
    # add column with the name of the video 
    df_temp['videoname'] = path.stem
   
    #export df for saving the downsampled data
    df_export = pd.DataFrame()
    
    # length of the total dataframe
    len_frames = len(df_temp)

    # set starting values for indexing
    start = 0
    end = fps
    
    # loop through data while we have not reached the end. This does skip frames that are not a whole second at the end
    while end <= len_frames:
        
        # randomly pick 20 frames, sort, append to big dataframe with name of file and export dataframe
        insert = df_temp.iloc[start:end].sample(n_frames).sort_index(axis=0)
        df_export = df_export.append(insert)
        df_big =  df_big.append(insert)

        # reset the index values to loop to the next data increment
        end += fps
        start += fps

    #save files
    filepath = input_dir/'Downsampled_data'
    filename = path.stem + '.csv'
    df_export.to_csv(filepath/filename,index = False) 

      bodypartscoords       nosex    nose.1y  nose.2likelihood     earLx.1  \
0                   0   -0.339909   3.387536          0.000021   -0.266320   
1                   1   -0.339909   3.387536          0.000021   -0.266320   
2                   2   -0.299916   3.329597          0.000020   -0.277875   
3                   3   -0.273909   3.342565          0.000019   -0.237432   
4                   4   -0.284779   3.344286          0.000019   -0.235054   
...               ...         ...        ...               ...         ...   
4881             4881  401.427490  73.964958          1.000000  380.285980   
4882             4882  394.601379  71.604233          1.000000  375.976776   
4883             4883  392.006653  71.485077          0.999999  373.877869   
4884             4884  390.964264  68.362831          1.000000  372.236633   
4885             4885  387.235931  64.297913          1.000000  370.195312   

       earL.1y.1  earL.2likelihood.1     earRx.2   earR.1y.2  \